In [1]:
import os
import torch
import pytorch_lightning as pl
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.models import resnet50
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.progress import RichProgressBar
import pandas as pd
from PIL import Image
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
from torchmetrics import AUROC, Accuracy, F1Score, Precision, Recall
from torchmetrics.classification import MulticlassROC
from sklearn.metrics import classification_report
import numpy as np
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from tabulate import tabulate
from sklearn.metrics import confusion_matrix
from torchvision.utils import make_grid

In [2]:
# Hiperparámetros
BATCH_SIZE = 32
NUM_EPOCHS = 100
LEARNING_RATE = 1e-4
NUM_WORKERS = 3
TOTAL_CLASSES = 6
CONFIDENCE_THRESHOLD = 0.60
IMAGE_SIZE = (224, 224)

In [3]:
# Rutas de los archivos Excel
excel_paths = [
    '/home/emontenegrob/dataset_frames_conf/dataset_frames_1_1.xlsx',
    '/home/emontenegrob/dataset_frames_conf/dataset_frames_2_2.xlsx',
    '/home/emontenegrob/dataset_frames_conf/dataset_frames_3_3.xlsx',
    '/home/emontenegrob/dataset_frames_conf/dataset_frames_4_4.xlsx',
    '/home/emontenegrob/dataset_frames_conf/dataset_frames_5_5.xlsx',
    '/home/emontenegrob/dataset_frames_conf/dataset_frames_6_6.xlsx'
]

In [4]:
class AnimalDataset(Dataset):
    def __init__(self, excel_paths, partition, transform=None):
        dataframes = [pd.read_excel(path) for path in excel_paths]
        self.data = pd.concat(dataframes, ignore_index=True)
        self.data = self.data[(self.data['Confianza'] > CONFIDENCE_THRESHOLD) & (self.data['Partition'] == partition)]
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_path = row['Frame (imagen luego del algoritmo de detección de movimiento)']
        bbox = row['Bounding Box']
        label = row['ID']
        
        image = Image.open(image_path)
        x1, y1, x2, y2 = eval(bbox)
        image = image.crop((x1, y1, x2, y2))

        if self.transform:
            image = self.transform(image)

        return image, label - 1


In [5]:
class AnimalDataModule(pl.LightningDataModule):
    def __init__(self, excel_paths, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS):
        super().__init__()
        self.excel_paths = excel_paths
        self.batch_size = batch_size
        self.num_workers = num_workers

    def setup(self, stage=None):
        self.train_transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.RandomCrop((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        self.test_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        self.train_dataset = AnimalDataset(self.excel_paths, partition='train', transform=self.train_transform)
        self.val_dataset = AnimalDataset(self.excel_paths, partition='validation', transform=self.test_transform)
        self.test_dataset = AnimalDataset(self.excel_paths, partition='test', transform=self.test_transform)
        
        print(f"Tamaño del conjunto de entrenamiento: {len(self.train_dataset)}")
        print(f"Tamaño del conjunto de validación: {len(self.val_dataset)}")
        print(f"Tamaño del conjunto de prueba: {len(self.test_dataset)}")

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)


In [6]:
def visualize_sample_images(data_module, logger, num_samples=2):
    """
    Visualiza 'num_samples' imágenes por clase del conjunto de entrenamiento y las registra en TensorBoard.
    
    Args:
    data_module (AnimalDataModule): El módulo de datos que contiene el dataset.
    logger (TensorBoardLogger): El logger de TensorBoard.
    num_samples (int): Número de muestras a visualizar por clase.
    """
    if not hasattr(data_module, 'train_dataset'):
        data_module.setup('fit')
    
    dataset = data_module.train_dataset
    class_samples = {i: [] for i in range(TOTAL_CLASSES)}
    
    for idx in range(len(dataset)):
        img, label = dataset[idx]
        if len(class_samples[label]) < num_samples:
            class_samples[label].append(img)
        
        if all(len(samples) == num_samples for samples in class_samples.values()):
            break
    
    # Crear un grid de imágenes para cada clase
    for class_idx, samples in class_samples.items():
        # Convertir la lista de imágenes a un tensor
        img_tensor = torch.stack(samples)
        
        # Crear un grid
        img_grid = make_grid(img_tensor, nrow=num_samples, padding=2, normalize=True)
        
        # Registrar el grid en TensorBoard
        logger.experiment.add_image(f'Sample Images Class {class_idx}', img_grid, dataformats='CHW')


In [7]:
def calculate_class_weights(data_module):
    labels = []
    for batch in data_module.train_dataloader():
        _, label = batch
        labels.extend(label.numpy())
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
    return torch.tensor(class_weights, dtype=torch.float32)


In [8]:
class LightningResNet50(pl.LightningModule):
    def __init__(self, learning_rate=LEARNING_RATE, num_classes=TOTAL_CLASSES, class_weights=None):
        super().__init__()
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        self.num_classes = num_classes
        self.class_weights = class_weights
        
        self.model = resnet50(pretrained=True)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)
        
        # Añadir estas líneas para inicializar las listas de historial
        self.train_loss_history = []
        self.val_loss_history = []
        self.train_acc_history = []
        self.val_acc_history = []

        # Congelar todas las capas excepto las últimas
        for param in self.model.parameters():
            param.requires_grad = False
        for param in self.model.layer4.parameters():
            param.requires_grad = True
        for param in self.model.fc.parameters():
            param.requires_grad = True

        self.train_acc = Accuracy(task="multiclass", num_classes=self.num_classes)
        self.val_acc = Accuracy(task="multiclass", num_classes=self.num_classes)
        self.test_acc = Accuracy(task="multiclass", num_classes=self.num_classes)

        self.train_f1 = F1Score(task="multiclass", num_classes=self.num_classes, average='macro')
        self.val_f1 = F1Score(task="multiclass", num_classes=self.num_classes, average='macro')
        self.test_f1 = F1Score(task="multiclass", num_classes=self.num_classes, average='macro')

        self.train_precision = Precision(task="multiclass", num_classes=self.num_classes, average='macro')
        self.val_precision = Precision(task="multiclass", num_classes=self.num_classes, average='macro')
        self.test_precision = Precision(task="multiclass", num_classes=self.num_classes, average='macro')

        self.train_recall = Recall(task="multiclass", num_classes=self.num_classes, average='macro')
        self.val_recall = Recall(task="multiclass", num_classes=self.num_classes, average='macro')
        self.test_recall = Recall(task="multiclass", num_classes=self.num_classes, average='macro')

        self.train_auc = AUROC(task="multiclass", num_classes=self.num_classes, average='macro')
        self.val_auc = AUROC(task="multiclass", num_classes=self.num_classes, average='macro')
        self.test_auc = AUROC(task="multiclass", num_classes=self.num_classes, average='macro')

        self.test_roc = MulticlassROC(num_classes=self.num_classes)

    def forward(self, x):
        return self.model(x)

    def _shared_step(self, batch):
        images, labels = batch
        class_weights = self.class_weights.to(self.device)
        logits = self(images)
        loss = nn.CrossEntropyLoss(weight=class_weights)(logits, labels)
        probs = torch.softmax(logits, dim=1)
        return loss, probs, labels

    def training_step(self, batch, batch_idx):
        loss, probs, labels = self._shared_step(batch)
        acc = self.train_acc(probs, labels)
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        self.log('train_acc', acc, on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        self.log('train_f1', self.train_f1(probs, labels), on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        self.log('train_precision', self.train_precision(probs, labels), on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        self.log('train_recall', self.train_recall(probs, labels), on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        self.log('train_auc', self.train_auc(probs, labels), on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, probs, labels = self._shared_step(batch)
        acc = self.val_acc(probs, labels)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        self.log('val_acc', acc, on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        self.log('val_f1', self.val_f1(probs, labels), on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        self.log('val_precision', self.val_precision(probs, labels), on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        self.log('val_recall', self.val_recall(probs, labels), on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        self.log('val_auc', self.val_auc(probs, labels), on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)

    def on_fit_start(self):
        # Registrar la estructura del modelo en TensorBoard
        self.logger.experiment.add_graph(self, torch.rand(1, 3, 224, 224).to(self.device))
        
        # Visualizar muestras de imágenes
        visualize_sample_images(self.trainer.datamodule, self.logger)
        
    def on_test_epoch_start(self):
        self.test_step_outputs = []
        
    def test_step(self, batch, batch_idx):
        loss, probs, labels = self._shared_step(batch)
        self.log('test_loss', loss, on_step=False, on_epoch=True)
        self.log('test_acc', self.test_acc(probs, labels), on_step=False, on_epoch=True)
        self.log('test_f1', self.test_f1(probs, labels), on_step=False, on_epoch=True)
        self.log('test_precision', self.test_precision(probs, labels), on_step=False, on_epoch=True)
        self.log('test_recall', self.test_recall(probs, labels), on_step=False, on_epoch=True)
        self.log('test_auc', self.test_auc(probs, labels), on_step=False, on_epoch=True)
        self.test_roc.update(probs, labels)
        
        output = {'loss': loss, 'probs': probs, 'labels': labels}
        self.test_step_outputs.append(output)
        return output
    
    def on_train_epoch_end(self):
        # Acumular los valores de pérdida y precisión
        train_loss = self.trainer.callback_metrics['train_loss'].cpu().item()
        val_loss = self.trainer.callback_metrics['val_loss'].cpu().item()
        train_acc = self.trainer.callback_metrics['train_acc'].cpu().item()
        val_acc = self.trainer.callback_metrics['val_acc'].cpu().item()

        self.train_loss_history.append(train_loss)
        self.val_loss_history.append(val_loss)
        self.train_acc_history.append(train_acc)
        self.val_acc_history.append(val_acc)

    def on_fit_end(self):
        # Crear gráficos de pérdida y precisión al final del entrenamiento
        epochs = range(len(self.train_loss_history))

        plt.figure(figsize=(12, 10))
        
        # Gráfico de pérdida
        plt.subplot(2, 1, 1)
        plt.plot(epochs, self.train_loss_history, 'b-', label='Train Loss')
        plt.plot(epochs, self.val_loss_history, 'r-', label='Validation Loss')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()

        # Gráfico de precisión
        plt.subplot(2, 1, 2)
        plt.plot(epochs, self.train_acc_history, 'b-', label='Train Accuracy')
        plt.plot(epochs, self.val_acc_history, 'r-', label='Validation Accuracy')
        plt.title('Training and Validation Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()

        plt.tight_layout()
        
        # Guardar la figura en TensorBoard
        self.logger.experiment.add_figure('Training History', plt.gcf())
        plt.close()


    def on_test_epoch_end(self):
        # Recopilación de todas las predicciones y etiquetas
        all_preds = []
        all_labels = []
        for output in self.test_step_outputs:
            all_preds.extend(output['probs'].cpu().numpy())
            all_labels.extend(output['labels'].cpu().numpy())

        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)

        # Binarizar las etiquetas
        all_labels_bin = label_binarize(all_labels, classes=list(range(self.num_classes)))
        
        # Calcular AUC-ROC para cada clase
        fpr = {}
        tpr = {}
        roc_auc = {}
        for i in range(self.num_classes):
            fpr[i], tpr[i], _ = roc_curve(all_labels_bin[:, i], all_preds[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        # Calcular AUC-ROC micro y macro
        fpr["micro"], tpr["micro"], _ = roc_curve(all_labels_bin.ravel(), all_preds.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        fpr["macro"] = np.unique(np.concatenate([fpr[i] for i in range(self.num_classes)]))
        tpr["macro"] = np.zeros_like(fpr["macro"])
        for i in range(self.num_classes):
            tpr["macro"] += np.interp(fpr["macro"], fpr[i], tpr[i])
        tpr["macro"] /= self.num_classes
        roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

        # Crear la gráfica ROC
        plt.figure(figsize=(14, 10))

        plt.plot(fpr["micro"], tpr["micro"],
                 label='Micro-average ROC curve (area = {0:0.2f})'
                       ''.format(roc_auc["micro"]),
                 color='deeppink', linestyle=':', linewidth=4)

        plt.plot(fpr["macro"], tpr["macro"],
                 label='Macro-average ROC curve (area = {0:0.2f})'
                       ''.format(roc_auc["macro"]),
                 color='navy', linestyle=':', linewidth=4)

        for i in range(self.num_classes):
            plt.plot(fpr[i], tpr[i], lw=2,
                     label='ROC curve of class {0} (area = {1:0.2f})'
                     ''.format(i, roc_auc[i]))

        plt.plot([0, 1], [0, 1], 'k--', lw=2)
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        
        plt.xlabel('False Positive Rate', fontsize=14)
        plt.ylabel('True Positive Rate', fontsize=14)
        plt.title('Receiver Operating Characteristic to Multi-Class', fontsize=16)
        plt.legend(loc="lower right", fontsize=10)

        # Guardar la figura en TensorBoard
        self.logger.experiment.add_figure('ROC Curve', plt.gcf())

        # Cerrar la figura para liberar memoria
        plt.close()

        # Calcular y registrar el classification report
        all_preds_class = all_preds.argmax(axis=1)
        report = classification_report(all_labels, all_preds_class, output_dict=True)
        
        # Convertir el reporte a un DataFrame
        df_report = pd.DataFrame(report).transpose()
        
        # Renombrar el índice para mayor claridad
        df_report.index.name = 'Class'
        df_report.reset_index(inplace=True)
        
        # Redondear los valores numéricos a 3 decimales
        df_report = df_report.round(3)
        
        # Crear una tabla estilizada
        table = tabulate(df_report, headers='keys', tablefmt='pretty', showindex=False)
        
        # Registrar la tabla en TensorBoard como texto
        self.logger.experiment.add_text('Classification Report', f'<pre>{table}</pre>')
        
        # También puedes guardar la tabla como una imagen si lo prefieres
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.axis('off')
        ax.table(cellText=df_report.values,
                 colLabels=df_report.columns,
                 cellLoc='center',
                 loc='center')
        plt.title('Classification Report')
        self.logger.experiment.add_figure('Classification Report Table', fig)
        plt.close(fig)

        # Visualizar el classification report como un heatmap
        plt.figure(figsize=(12, 8))
        sns.heatmap(df_report.iloc[:, 1:5], annot=True, cmap='YlGnBu')
        plt.title('Classification Report Heatmap')
        self.logger.experiment.add_figure('Classification Report Heatmap', plt.gcf())
        plt.close()
        
        # Calcular la matriz de confusión
        cm = confusion_matrix(all_labels, all_preds_class)
        
        # Crear una figura para la matriz de confusión
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted Labels')
        plt.ylabel('True Labels')
        self.logger.experiment.add_figure('Confusion Matrix', plt.gcf())
        plt.close()

        # Calcular la matriz de confusión normalizada
        cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
        # Crear una figura para la matriz de confusión normalizada
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Blues')
        plt.title('Normalized Confusion Matrix')
        plt.xlabel('Predicted Labels')
        plt.ylabel('True Labels')
        self.logger.experiment.add_figure('Normalized Confusion Matrix', plt.gcf())
        plt.close()

        # Limpiar los outputs para la siguiente época
        self.test_step_outputs.clear()

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def on_fit_start(self):
        # Registrar la estructura del modelo en TensorBoard
        self.logger.experiment.add_graph(self, torch.rand(1, 3, 224, 224).to(self.device))


In [9]:
# Inicializar el DataModule
data_module = AnimalDataModule(excel_paths)
data_module.setup(stage='fit')

# Calcular los pesos de clase
class_weights = calculate_class_weights(data_module)

Tamaño del conjunto de entrenamiento: 20931
Tamaño del conjunto de validación: 3796
Tamaño del conjunto de prueba: 4705


In [10]:
# Configurar el logger de TensorBoard
logger = TensorBoardLogger("tb_logs_resnet50", name="resnet50_animals_jocotoco_conf60")

# Inicializar el modelo
lightning_model = LightningResNet50(learning_rate=LEARNING_RATE, num_classes=TOTAL_CLASSES, class_weights=class_weights)

# Configurar los callbacks
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='checkpoints_resnet50',
    filename='resnet50-{epoch:02d}-{val_loss:.2f}',
    save_top_k=3,
    mode='min'
)

early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=10,
    mode='min'
)

progress_bar_callback = RichProgressBar()

# Configurar el Trainer
trainer = pl.Trainer(
    max_epochs=NUM_EPOCHS,
    callbacks=[checkpoint_callback, early_stopping_callback, progress_bar_callback],
    accelerator='gpu',
    devices=1,
    logger=logger,
    log_every_n_steps=50,
    deterministic=True
)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [11]:
# Entrenamiento
trainer.fit(lightning_model, datamodule=data_module)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /home/emontenegrob/codigo_solo_tesis_resnet50_testq_conf60/checkpoints_resnet50 exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Tamaño del conjunto de entrenamiento: 20931
Tamaño del conjunto de validación: 3796
Tamaño del conjunto de prueba: 4705


┏━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃    ┃ Name            ┃ Type                ┃ Params ┃ Mode  ┃
┡━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0  │ model           │ ResNet              │ 23.5 M │ train │
│ 1  │ train_acc       │ MulticlassAccuracy  │      0 │ train │
│ 2  │ val_acc         │ MulticlassAccuracy  │      0 │ train │
│ 3  │ test_acc        │ MulticlassAccuracy  │      0 │ train │
│ 4  │ train_f1        │ MulticlassF1Score   │      0 │ train │
│ 5  │ val_f1          │ MulticlassF1Score   │      0 │ train │
│ 6  │ test_f1         │ MulticlassF1Score   │      0 │ train │
│ 7  │ train_precision │ MulticlassPrecision │      0 │ train │
│ 8  │ val_precision   │ MulticlassPrecision │      0 │ train │
│ 9  │ test_precision  │ MulticlassPrecision │      0 │ train │
│ 10 │ train_recall    │ MulticlassRecall    │      0 │ train │
│ 11 │ val_recall      │ MulticlassRecall    │      0 │ train │
│ 12 │ test_recall     │ MulticlassRecall    │      0 │ train │
│ 13 │ train_auc       │ MulticlassAUROC     │      0 │ train │
│ 14 │ val_auc         │ MulticlassAUROC     │      0 │ train │
│ 15 │ test_auc        │ MulticlassAUROC     │      0 │ train │
│ 16 │ test_roc        │ MulticlassROC       │      0 │ train │
└────┴─────────────────┴─────────────────────┴────────┴───────┘

Trainable params: 15.0 M                                                                                           
Non-trainable params: 8.5 M                                                                                        
Total params: 23.5 M                                                                                               
Total estimated model params size (MB): 94                                                                         
Modules in train mode: 167                                                                                         
Modules in eval mode: 0

Output()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is 
incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()

/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: TorchMetricsUserWarning: You are 
trying to use a metric in deterministic mode on GPU that uses `torch.cumsum`, which is currently not supported. The
tensor will be copied to the CPU memory to compute it and then copied back to GPU. Expect some slowdowns.
  warnings.warn(*args, **kwargs)  # noqa: B028

/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No negative samples in 
targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)  # noqa: B028

/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in 
targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028

In [12]:
# Cargar el mejor modelo
best_model_path = checkpoint_callback.best_model_path
best_model = LightningResNet50.load_from_checkpoint(best_model_path)
print(f"Mejor modelo guardado en: {best_model_path}")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Mejor modelo guardado en: /home/emontenegrob/codigo_solo_tesis_resnet50_testq_conf60/checkpoints_resnet50/resnet50-epoch=18-val_loss=0.09.ckpt


In [13]:
# Evaluación en el conjunto de prueba
trainer.test(model=best_model, datamodule=data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Tamaño del conjunto de entrenamiento: 20931
Tamaño del conjunto de validación: 3796
Tamaño del conjunto de prueba: 4705


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Output()

/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: TorchMetricsUserWarning: You are 
trying to use a metric in deterministic mode on GPU that uses `torch.cumsum`, which is currently not supported. The
tensor will be copied to the CPU memory to compute it and then copied back to GPU. Expect some slowdowns.
  warnings.warn(*args, **kwargs)  # noqa: B028

/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No negative samples in 
targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)  # noqa: B028

/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in 
targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │     0.950478196144104     │
│         test_auc          │    0.01129637099802494    │
│          test_f1          │    0.8366580009460449     │
│         test_loss         │    0.14273853600025177    │
│      test_precision       │    0.8500422239303589     │
│        test_recall        │    0.8287150859832764     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.14273853600025177,
  'test_acc': 0.950478196144104,
  'test_f1': 0.8366580009460449,
  'test_precision': 0.8500422239303589,
  'test_recall': 0.8287150859832764,
  'test_auc': 0.01129637099802494}]

In [1]:
# Visualización de TensorBoard (si estás ejecutando en un notebook)
%load_ext tensorboard
%tensorboard --logdir tb_logs_resnet50

<IPython.core.display.Javascript object>